# QuantLib basics {#quantlib-basics}

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} -c martinbrose quantlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/custom_python

  added / updated specs:
    - quantlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    quantlib-1.21              |           py36_0        15.6 MB  martinbrose
    ------------------------------------------------------------
                                           Total:        15.6 MB

The following NEW packages will be INSTALLED:

  quantlib           martinbrose/linux-64::quantlib-1.21-py36_0



quantlib-1.21        | 15.6 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In this chapter we will introduce some of the basic concepts such as `Date`, `Period`, `Calendar` and `Schedule`. These are QuantLib constructs that are used throughout the library in creation of instruments, models, term structures etc.

In [3]:
import QuantLib as ql
import pandas as pd

#### Date Class

The `Date` object can be created using the constructor as `Date(day, month, year)`. It would be worthwhile to pay attention to the fact that `day` is the first argument, followed by `month` and then the `year`. This is different from the Python `datetime` object instantiation.

In [8]:
date = ql.Date(25, 10, 2021)
print(date)

October 25th, 2021


The fields of the `Date` object can be accessed using the `month()`, `dayOfMonth()` and `year()` methods. The `weekday()` method can be used to fetch the day of the week. 

In [9]:
print("%d-%d-%d" %(date.month(), 
                   date.dayOfMonth(),
                   date.year()))

10-25-2021


In [10]:
date.weekday() == ql.Tuesday

False

The `Date` objects can also be used to perform arithmetic operations such as advancing by days, weeks, months etc. Periods such as weeks or months can be denoted using the `Period` class. `Period` object constructor signature is `Period(num_periods, period_type)`. The `num_periods` is an integer and represents the number of periods. The `period_type` can be `Weeks`, `Months` and `Years`.

In [11]:
type(date+1)

QuantLib.QuantLib.Date

In [12]:
print("Add a day      : {0}".format(date + 1))
print("Subtract a day : {0}".format(date - 1))
print("Add a week     : {0}".format(date + ql.Period(1, ql.Weeks)))
print("Add a month    : {0}".format(date + ql.Period(1, ql.Months)))
print("Add a year     : {0}".format(date + ql.Period(1, ql.Years)))

Add a day      : October 26th, 2021
Subtract a day : October 24th, 2021
Add a week     : November 1st, 2021
Add a month    : November 25th, 2021
Add a year     : October 25th, 2022


One can also do logical operations using the `Date` object.

In [13]:
print(date == ql.Date(31, 3, 2015))
print(date > ql.Date(30, 3, 2015))
print(date < ql.Date(1, 4, 2015))
print(date != ql.Date(1, 4, 2015))

False
True
False
True


The `Date` object is used in setting valuation dates, issuance and expiry dates of instruments. The `Period` object is used in setting tenors, such as that of coupon payments, or in constructing payment schedules.

#### Calendar Class

The `Date` arithmetic above did not take holidays into account. But valuation of different securities would require taking into account the holidays observed in a specific exchange or country. The `Calendar` class implements this functionality for all the major exchanges. Let us take a look at a few examples here. 

In [17]:
date = ql.Date(31, 3, 2015)
us_calendar = ql.UnitedStates()
italy_calendar = ql.Italy()
jp_calendar = ql.Japan()
cn_calendar = ql.China()

period = ql.Period(60, ql.Days)
raw_date = date + period
us_date = us_calendar.advance(date, period)
italy_date = italy_calendar.advance(date, period)
jp_date = jp_calendar.advance(date, period)
cn_date = cn_calendar.advance(date, period)

print("Add 60 days: {0}".format(raw_date))
print("Add 60 business days in US: {0}".format(us_date))
print("Add 60 business days in Italy: {0}".format(italy_date))
print("Add 60 business days in Japan: {0}".format(jp_date))
print("Add 60 business days in China: {0}".format(cn_date))

Add 60 days: May 30th, 2015
Add 60 business days in US: June 24th, 2015
Add 60 business days in Italy: June 26th, 2015
Add 60 business days in Japan: June 29th, 2015
Add 60 business days in China: June 26th, 2015


The `addHoliday` and `removeHoliday` methods in the calendar can be used to add and remove holidays to the calendar respectively. If a calendar has any missing holidays or has a wrong holiday, then these methods come handy in fixing the errors. The `businessDaysBetween` method helps find out the number of business days between two dates per a given calendar. Let us use this method on the `us_calendar` and `italy_calendar` as a sanity check.

In [18]:
us_busdays = us_calendar.businessDaysBetween(date, us_date)
italy_busdays = italy_calendar.businessDaysBetween(date, italy_date)
jp_busdays = jp_calendar.businessDaysBetween(date, jp_date)
cn_busdays = cn_calendar.businessDaysBetween(date, cn_date)

print("Business days US: {0}".format(us_busdays))
print("Business days Italy: {0}".format(italy_busdays))
print("Business days Japan: {0}".format(jp_busdays))
print("Business days China: {0}".format(cn_busdays))

Business days US: 60
Business days Italy: 60
Business days Japan: 60
Business days China: 60


In valuation of certain deals, more than one calendar's holidays are observed. QuantLib has `JointCalendar` class that allows you to combine the holidays of two or more calendars. Let us take a look at a working example.

In [19]:
joint_calendar = ql.JointCalendar(us_calendar, italy_calendar)

joint_date = joint_calendar.advance(date, period)
joint_busdays = joint_calendar.businessDaysBetween(date, joint_date)

print("Add 60 business days in US-Italy: {0}".format(joint_date))
print("Business days US-Italy: {0}".format(joint_busdays))

Add 60 business days in US-Italy: June 29th, 2015
Business days US-Italy: 60


#### Schedule Class

The `Schedule` object is necessary in creating coupon schedules or call schedules. `Schedule` object constructors have the following signature:

    
    Schedule(const Date& effectiveDate,
             const Date& terminationDate,
             const Period& tenor,
             const Calendar& calendar,
             BusinessDayConvention convention,
             BusinessDayConvention terminationDateConvention,
             DateGeneration::Rule rule,
             bool endOfMonth,
             const Date& firstDate = Date(),
             const Date& nextToLastDate = Date())
    

and 


    Schedule(const std::vector<Date>&,
             const Calendar& calendar,
             BusinessDayConvention rollingConvention)



In [21]:
effective_date = ql.Date(1, 1, 2015)
termination_date = ql.Date(1, 1, 2016)
tenor = ql.Period(ql.Monthly)
calendar = ql.UnitedStates()
business_convention = ql.Following
termination_business_convention = ql.Following
date_generation = ql.DateGeneration.Forward
end_of_month = False

schedule = ql.Schedule(effective_date,
                       termination_date,
                       tenor,
                       calendar,
                       business_convention,
                       termination_business_convention,
                       date_generation,
                       end_of_month)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


Here we have generated a `Schedule` object that will contain dates between `effective_date` and `termination_date` with the `tenor` specifying the `Period` to be `Monthly`. The `calendar` object is used for determining holidays. Here we have chosen the convention to be the day following holidays. That is why we see that holidays are excluded in the list of dates.



The `Schedule` class can handle generation of dates with irregularity in schedule. The two extra parameters `firstDate` and `nextToLastDate` parameters along with a combination of forward or backward date generation rule can be used to generate short or long stub payments at the front or back end of the schedule. For example, the following combination of `firstDate` and backward generation rule creates a short stub in the front on the January 15, 2015.

In [22]:
# short stub in the front
effective_date = ql.Date(1, 1, 2015)
termination_date = ql.Date(1, 1, 2016)
first_date = ql.Date(15, 1, 2015)
schedule = ql.Schedule(effective_date,
                       termination_date,
                       tenor,
                       calendar,
                       business_convention,
                       termination_business_convention,
                       ql.DateGeneration.Backward,
                       end_of_month,
                       first_date)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"January 15th, 2015"
2,"February 2nd, 2015"
3,"March 2nd, 2015"
4,"April 1st, 2015"
5,"May 1st, 2015"
6,"June 1st, 2015"
7,"July 1st, 2015"
8,"August 3rd, 2015"
9,"September 1st, 2015"


Using the `nextToLastDate` parameter along with the forward date generation rule creates a short stub at the back end of the schedule.

In [23]:
# short stub at the back
effective_date = ql.Date(1, 1, 2015)
termination_date = ql.Date(1, 1, 2016)
penultimate_date = ql.Date(15, 12, 2015)
schedule = ql.Schedule(effective_date,
                       termination_date,
                       tenor,
                       calendar,
                       business_convention,
                       termination_business_convention,
                       ql.DateGeneration.Forward,
                       end_of_month,
                       ql.Date(),
                       penultimate_date)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


Using the backward generation rule along with the `firstDate` allows us to create a long stub in the front. Below the first two dates are longer in duration than the rest of the dates.

In [25]:
# long stub in the front
first_date = ql.Date(1, 2, 2015)
effective_date = ql.Date(15, 12, 2014)
termination_date = ql.Date(1, 1, 2016)
schedule = ql.Schedule(effective_date,
                       termination_date,
                       tenor,
                       calendar,
                       business_convention,
                       termination_business_convention,
                       ql.DateGeneration.Backward,
                       end_of_month, 
                       first_date)

pd.DataFrame({'date': list(schedule)})

,date
0,"December 15th, 2014"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


Similarly the usage of `nextToLastDate` parameter along with forward date generation rule can be used to generate long stub at the back of the schedule.

In [26]:
# long stub at the back
effective_date = ql.Date(1, 1, 2015)
penultimate_date = ql.Date(1, 12, 2015)
termination_date = ql.Date(15, 1, 2016)
schedule = ql.Schedule(effective_date,
                       termination_date,
                       tenor,
                       calendar,
                       business_convention,
                       termination_business_convention,
                       ql.DateGeneration.Forward,
                       end_of_month,
                       ql.Date(),
                       penultimate_date)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


Below the `Schedule` is generated from a list of dates.

In [16]:
dates = [ql.Date(2,1,2015), ql.Date(2, 2,2015),
         ql.Date(2,3,2015), ql.Date(1,4,2015),
         ql.Date(1,5,2015), ql.Date(1,6,2015),
         ql.Date(1,7,2015), ql.Date(3,8,2015),
         ql.Date(1,9,2015), ql.Date(1,10,2015),
         ql.Date(2,11,2015), ql.Date(1,12,2015),
         ql.Date(4,1,2016)]
rolling_convention = ql.Following

schedule = ql.Schedule(dates, calendar,
                       rolling_convention)

pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"February 2nd, 2015"
2,"March 2nd, 2015"
3,"April 1st, 2015"
4,"May 1st, 2015"
5,"June 1st, 2015"
6,"July 1st, 2015"
7,"August 3rd, 2015"
8,"September 1st, 2015"
9,"October 1st, 2015"


#### Interest Rate

The `InterestRate` class can be used to store the interest rate with the compounding type, day count and
the frequency of compounding. Below we show how to create an interest rate of 5.0% compounded annually,
using Actual/Actual day count convention.

In [27]:
annual_rate = 0.05
day_count = ql.ActualActual()
compound_type = ql.Compounded
frequency = ql.Annual

interest_rate = ql.InterestRate(annual_rate, 
                                day_count, 
                                compound_type, 
                                frequency)
print(interest_rate)

5.000000 % Actual/Actual (ISDA) Annual compounding


Lets say if you invest a dollar at the interest rate described by `interest_rate`, the
`compoundFactor` method in the `InterestRate` object gives you how much your investment will be worth after any period.
Below we show that the value returned by ``compound_factor`` for 2 years agrees with
the expected compounding formula.

In [28]:
t = 2.0
print(interest_rate.compoundFactor(t))
print((1+annual_rate)*(1.0+annual_rate))

1.1025
1.1025


The `discountFactor` method returns the reciprocal of the `compoundFactor` method.
The discount factor is useful while calculating the present value of future cashflows.

In [29]:
print(interest_rate.discountFactor(t))
print(1.0/interest_rate.compoundFactor(t))

0.9070294784580498
0.9070294784580498


A given interest rate can be converted into other compounding types and compounding frequency using the `equivalentRate` method.

In [30]:
new_frequency = ql.Semiannual
new_interest_rate = interest_rate.equivalentRate(compound_type,
                                                 new_frequency, t)
print(new_interest_rate)

4.939015 % Actual/Actual (ISDA) Semiannual compounding


The discount factor for the two `InterestRate` objects, `interest_rate` and `new_interest_rate` are the same, as shown below.

In [31]:
print(interest_rate.discountFactor(t))
print(new_interest_rate.discountFactor(t))

0.9070294784580498
0.9070294784580495


The `impliedRate` method in the `InterestRate` class
takes compound factor to return the implied rate. The ``impliedRate`` method
is a static method in the ``InterestRate`` class and can be used without an
instance of ``InterestRate``. Internally the ``equivalentRate`` method invokes
the ``impliedRate`` method in its calculations.

#### Conclusion

This chapter gave an introduction to the basics of QuantLib. Here we explained the `Date`, `Schedule`, `Calendar` and `InterestRate` classes.

In [32]:
from pandas_datareader import data as pdr

data = pdr.get_data_yahoo('MSFT',start='2020-09-25', end='2020-10-02')
print(data)

                  High         Low        Open       Close    Volume  \
Date                                                                   
2020-09-25  209.039993  202.539993  203.550003  207.820007  29437300   
2020-09-28  212.570007  208.059998  210.880005  209.440002  32004900   
2020-09-29  210.070007  206.809998  209.350006  207.259995  24221900   
2020-09-30  211.979996  206.539993  207.729996  210.330002  33829100   
2020-10-01  213.990005  211.320007  213.490005  212.460007  27158400   
2020-10-02  210.990005  205.539993  208.000000  206.190002  33154800   

             Adj Close  
Date                    
2020-09-25  205.930389  
2020-09-28  207.535645  
2020-09-29  205.375473  
2020-09-30  208.417557  
2020-10-01  210.528198  
2020-10-02  204.315201  
